# Funciones 2.0

---
Funciones como argumentos de otras funciones

In [1]:
def greeting(name):
    print(f"Hello {name}")

In [2]:
def explained_greeting(greeting_fn, name):
    print("This is an special greeting from your loved one")
    greeting_fn(name)

In [3]:
explained_greeting(greeting, "Cata")

This is an special greeting from your loved one
Hello Cata


---
Funciones que retornan funciones

In [4]:
def make_greeting_function(name):
    def fn():
        print(f"Hello {name}")
    return fn

In [5]:
greeting_fn = make_greeting_function("Cata")

In [6]:
greeting_fn()

Hello Cata


---
Funciones que reciben funciones y entregan una versión extendida de la original

In [7]:
def greet_with_pleasure(greeting_fn):
    def helper():
        greeting_fn()
        print("It is a pleasure")
    return helper

In [8]:
greeting_fn2 = greet_with_pleasure(greeting_fn)
greeting_fn2()

Hello Cata
It is a pleasure


---
Hay otra sintaxis para esto

In [9]:
def call_two_times(fn):
    def helper(name):
        fn(name)
        fn(name)
    return helper

In [10]:
@call_two_times
def greeting(name):
    print(f"Hello {name}")

In [11]:
greeting("Cata")

Hello Cata
Hello Cata


---
Funciones que se llaman a si mismas (recursividad)

In [12]:
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [13]:
[fib(n) for n in range(16)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

Python ofrece una forma de guardar en memoria resultados frecuentes, esto nos permite optimizar la velocidad de nuestras funciones, utilizando un poco más de memoria RAM

In [14]:
import functools

In [15]:
@functools.lru_cache(maxsize=None)
def fib2(n):
    if n < 2:
        return n
    return fib2(n-1) + fib2(n-2)

In [16]:
%timeit [fib(n) for n in range(16)]

542 µs ± 68.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
%timeit [fib2(n) for n in range(16)]

1.48 µs ± 21.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


**Reflexión**

¿cómo crees que fue implementada la funcicón `lru_cache`?

## Mas functools

El módulo `functools` nos ofrece varias funciones útiles que reciben como argumento otras funciones, así como `functools.lru_cache`. Siempre podrás leer más en la [documentación oficial](https://docs.python.org/3.7/library/functools.html)

In [18]:
import functools

[**partial**](https://docs.python.org/3.7/library/functools.html#functools.partial)

Imagina que tenemos la siguiente función

In [19]:
def print_student_info(name, email, degree, university):
    template ="""
        Name          : {}
        Email         : {}
        Degree        : {}
        University    : {}
        """
    print(template.format(name, email, degree, university))

In [20]:
print_student_info("Danilo", "ddiazvaxx@gmail.com", "Mechanical Engineer", "EAFIT")


        Name          : Danilo
        Email         : ddiazvaxx@gmail.com
        Degree        : Mechanical Engineer
        University    : EAFIT
        


Ahora queremos una función que nos sirva para imprimir a todos los estudiantes que hayan realizado su pregrado en EAFIT

In [21]:
print_eafit_student_info = functools.partial(print_student_info, university="EAFIT")

In [22]:
print_eafit_student_info("Julian", "jeussejxx@gmail.com", "Mechanical Engineer")


        Name          : Julian
        Email         : jeussejxx@gmail.com
        Degree        : Mechanical Engineer
        University    : EAFIT
        


Usando `functools.partial` sobre una función `f` podemos crear una función `f` que funcione tal como `f`, solo que tiene un argumento fijo. Esto potencialmente eliminaría errores pasando los mismos argumentos una y otra vez.

[**reduce**](https://docs.python.org/3.7/library/functools.html#functools.reduce)

> Apply function of two arguments cumulatively to the items of sequence, from left to right, so as to reduce the sequence to a single value

> The left argument, x, is the accumulated value and the right argument, y, is the update value from the sequence. If the optional initializer is present, it is placed before the items of the sequence in the calculation, and serves as a default when the sequence is empty. If initializer is not given and sequence contains only one item, the first item is returned.

Ahora un ejempo para implementar una productoria de una lista de números

In [23]:
def product(seq):
    return functools.reduce(lambda x, y: x*y, seq)

In [24]:
product([1,2,3,4])

24

[**singledispatch**](https://docs.python.org/3.7/library/functools.html#functools.singledispatch)

Imaginemos que tenemos una función que recibe un argumento y dependiendo del tipo de dato del argumento realiza una tarea, como se muestra a continuación

In [25]:
def compute_total_string_length(value):
    if isinstance(value, str):
        return len(value)
    elif isinstance(value, list):
        return sum(len(v) for v in value)
    elif isinstance(value, int):
        return len(str(value))
    else:
        raise TypeError(f"unsupported argument {type(value)}")

In [26]:
print(compute_total_string_length("hola!"))
print(compute_total_string_length(["hola!", "a", "todos", "los", "estudiantes"]))
print(compute_total_string_length(456))

5
25
3


esta misma función se hubiera podido escribir de la siguiente manera

In [27]:
@functools.singledispatch
def compute_total_string_length(value):
    raise TypeError(f"unsupported argument {type(value)}")

In [28]:
@compute_total_string_length.register
def _(value: str):
    return len(value)

@compute_total_string_length.register
def _(value: int):
    return len(str(value))

@compute_total_string_length.register
def _(value: list):
    return sum(len(v) for v in value)

In [29]:
print(compute_total_string_length("hola!"))
print(compute_total_string_length(["hola!", "a", "todos", "los", "estudiantes"]))
print(compute_total_string_length(456))

5
25
3


En conclusión, esta función nos permite ahorrarnos esos *feos* condicionales, que para el ejemplo mostrado no parece la gran cosa, pero para funciones más complicadas nos permite tener un código más fácil de leer.

[**wrapper**](https://docs.python.org/3.7/library/functools.html#functools.wraps)

Esta nos permite que una función que pasa por un decorador se parezca más a la función decorada. Vamos a ilustrarlo implementando nuestra propia versión de `functools.lru_cache`, la cual seguramente no será tan chévere.

In [30]:
def lru_cache(f):
    cache = {}
    def wrapper(v):
        if v in cache:
            return cache[v]
        out = f(v)
        cache[v] = out
        return out
    return wrapper

Volvamos de nuevo a nuestra función para calcular los números de fibonacci para cierto `n`

In [31]:
@lru_cache
def fib(n: int):
    """Computes the Fibonacci number at index n"""
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [32]:
[fib(n) for n in range(16)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

Funciona como esperamos, pero miremos lo que pasa si tratamos de consultar la documentación

In [33]:
help(fib)

Help on function wrapper in module __main__:

wrapper(v)



Perdimos nuestra documentación! Ahora miremos que pasa si utilizamos `functools.wraps`

In [34]:
def lru_cache(f):
    cache = {}
    @functools.wraps(f)
    def wrapper(v):
        if v in cache:
            return cache[v]
        out = f(v)
        cache[v] = out
        return out
    return wrapper

In [35]:
@lru_cache
def fib(n: int):
    """Computes the Fibonacci number at index n"""
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [36]:
help(fib)

Help on function fib in module __main__:

fib(n: int)
    Computes the Fibonacci number at index n



:) 